In [ ]:
# PSEUDOCODE / DETAILED PLAN:
# 1. Import the target function using an absolute import:
#      from apprentissage.quiSeraLeVainqueur import quiSeraLeVainqueur
# 2. Locate the pokedex.csv file from the repository to obtain a list of Pokemon names.
#    - Try common relative locations: "./apprentissage/data/pokedex.csv", "./data/pokedex.csv"
#    - Read the CSV header and pick a reasonable text column to extract names (try common headers).
#    - Fall back to reading the second column of the CSV if header names are unknown.
# 3. Limit the number of Pokemon to a small N (configurable, e.g. 8) to keep test runtime short.
# 4. Generate all unordered pairs of distinct Pokemon from that list (combinations).
# 5. For each pair (A, B), call quiSeraLeVainqueur in a flexible way:
#    - Try quiSeraLeVainqueur(A, B)
#    - If TypeError, try quiSeraLeVainqueur([A, B]) or quiSeraLeVainqueur(A, B, model_path) if model arg exists
#    - If the function raises any unexpected exception, fail the test with clear diagnostics.
# 6. Collect the winners into a tally mapping winner -> count.
# 7. Assert the following test conditions:
#    - No unexpected exceptions occurred while querying all pairs.
#    - The number of results equals the number of pairs.
#    - Every reported winner is one of the two participants for that match (or at least a non-empty string).
#    - There is at least one winner (top count >= 1).
# 8. Print a short summary of the top winners to the notebook output for inspection.
#
# The resulting notebook cell(s) are runnable as tests: they will raise AssertionError on failure.

# TEST CODE
from apprentissage.quiSeraLeVainqueur import quiSeraLeVainqueur

import csv
import itertools
import os
from pathlib import Path
from collections import Counter
import inspect

# Helper: find pokedex.csv in likely paths
def find_pokedex():
    candidates = [
        Path("apprentissage") / "data" / "pokedex.csv",
        Path("apprentissage") / "pokedex.csv",
        Path("data") / "pokedex.csv",
        Path("apprentissage") / "data" / "pokedex.csv",
        Path("data") / "pokedex.csv",
    ]
    for p in candidates:
        if p.exists():
            return p
    # fallback: search recursively
    for p in Path(".").rglob("pokedex.csv"):
        return p
    raise FileNotFoundError("pokedex.csv not found in expected locations")

pokedex_path = find_pokedex()

# Read names from pokedex.csv
names = []
with pokedex_path.open(newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    headers = next(reader, None)
    # Try common header names
    # If headers contain a known column, use it; otherwise use second column index 1 if available
    name_index = None
    if headers:
        lower = [h.lower() for h in headers]
        for candidate in ("name", "nom", "pokemon", "poke"):
            if candidate in lower:
                name_index = lower.index(candidate)
                break
    if name_index is None:
        # fall back to index 1 if there are at least 2 columns, else index 0
        name_index = 1 if len(headers) > 1 else 0
        # put header back as first data row if we used it as data
        if headers:
            # try to use header row as data only if it looks like a name (non-numeric)
            maybe = headers[name_index] if len(headers) > name_index else None
            if maybe and not maybe.isdigit():
                # include header value as first name if it seems like a name
                names.append(maybe)
    # Now read remaining rows
    for row in reader:
        if len(row) > name_index:
            val = row[name_index].strip()
            if val:
                names.append(val)

# Ensure we have some names
assert len(names) >= 2, f"Need at least 2 Pokémon in pokedex.csv, found {len(names)}"

# Limit number of Pokemon for test speed (adjust N as needed)
N = 8
names = list(dict.fromkeys(names))[:N]  # preserve order, unique, take first N

pairs = list(itertools.combinations(names, 2))
expected_pairs_count = len(pairs)
assert expected_pairs_count > 0

# Function invocation helper: try a few calling conventions
def call_predict(a, b):
    # Try several calling patterns based on introspection
    sig = None
    try:
        sig = inspect.signature(quiSeraLeVainqueur)
    except Exception:
        sig = None
    # 1) try two positional args
    try:
        return quiSeraLeVainqueur(a, b)
    except TypeError:
        pass
    # 2) try single list arg
    try:
        return quiSeraLeVainqueur([a, b])
    except TypeError:
        pass
    # 3) try keyword names if signature known
    if sig:
        params = list(sig.parameters)
        kwargs = {}
        if "pair" in params or "pokemons" in params or "pokemon_pair" in params:
            key = next((k for k in params if k in ("pair","pokemons","pokemon_pair")), None)
            try:
                return quiSeraLeVainqueur(**{key: [a,b]})
            except Exception:
                pass
        if "a" in params and "b" in params:
            try:
                return quiSeraLeVainqueur(a=a, b=b)
            except Exception:
                pass
    # 4) As a last resort, try converting names to simple ids or lowercased strings
    try:
        return quiSeraLeVainqueur(str(a), str(b))
    except Exception as e:
        raise AssertionError(f"quiSeraLeVainqueur failed for pair ({a},{b}): {e}")

# Run all pairwise predictions and tally winners
tally = Counter()
results = {}
for a, b in pairs:
    try:
        winner = call_predict(a, b)
    except AssertionError:
        # re-raise to fail the test with diagnostic
        raise
    except Exception as e:
        raise AssertionError(f"Unexpected exception for pair ({a},{b}): {e}")
    # Normalize winner if it's a tuple/list/dict etc.
    if isinstance(winner, (list, tuple)):
        # take first element if a sequence
        winner_val = winner[0] if len(winner) else ""
    elif isinstance(winner, dict):
        # take first value
        winner_val = next(iter(winner.values())) if winner else ""
    else:
        winner_val = winner
    if winner_val is None:
        winner_val = ""
    winner_str = str(winner_val).strip()
    # Basic validation: winner should be one of the participants OR a non-empty string
    if winner_str:
        # It's acceptable for model to return a label different from input (e.g., an ID or normalized name).
        # We only assert that it is non-empty and is string-like.
        pass
    else:
        raise AssertionError(f"Empty winner returned for pair ({a},{b})")
    tally[winner_str] += 1
    results[(a,b)] = winner_str

# Assertions per plan
# - Number of recorded results equals number of pairs
assert len(results) == expected_pairs_count, f"Expected {expected_pairs_count} results, got {len(results)}"

# - There is at least one winner with at least 1 victory
top_winner, top_count = tally.most_common(1)[0]
assert top_count >= 1, "Top winner has zero wins (unexpected)"

# Test passed: print a concise summary for user inspection
print(f"Tested {expected_pairs_count} matches among {len(names)} Pokémon.")
print(f"Top winner: {top_winner!r} with {top_count} wins.")
print("Top 5 winners:", tally.most_common(5))